In [57]:
from pyspark.sql import SparkSession, functions as F
import pandas as pd
import json

In [58]:
spark = SparkSession\
  .builder\
  .master("local[*]")\
  .appName('test')\
  .config("spark.driver.bindAddress", "127.0.0.1")\
  .getOrCreate()

In [ ]:
matches_df_nested = spark.read\
  .option("multiline", "true")\
  .json("./data/matches/*/*")

In [ ]:
matches_df_nested.printSchema()

root
 |-- away_score: long (nullable = true)
 |-- away_team: struct (nullable = true)
 |    |-- away_team_gender: string (nullable = true)
 |    |-- away_team_group: string (nullable = true)
 |    |-- away_team_id: long (nullable = true)
 |    |-- away_team_name: string (nullable = true)
 |    |-- country: struct (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |-- managers: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- country: struct (nullable = true)
 |    |    |    |    |-- id: long (nullable = true)
 |    |    |    |    |-- name: string (nullable = true)
 |    |    |    |-- dob: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- name: string (nullable = true)
 |    |    |    |-- nickname: string (nullable = true)
 |-- competition: struct (nullable = true)
 |    |-- competition_id: long (nullable = true)
 |    |-- competition_nam

In [ ]:
matches_df_flat = matches_df_nested \
  .select([
    'match_id',
    'match_date',
    'competition.competition_name',
    'competition_stage.name',
    'season.season_name',
    'match_week',
    'kick_off',
    'home_team.home_team_name',
    'away_team.away_team_name',
    'home_score',
    'away_score',
    'last_updated'
  ])

In [ ]:
matches_df_flat.show()

+--------+----------+----------------+--------------+-----------+----------+------------+--------------------+--------------------+----------+----------+--------------------+
|match_id|match_date|competition_name|          name|season_name|match_week|    kick_off|      home_team_name|      away_team_name|home_score|away_score|        last_updated|
+--------+----------+----------------+--------------+-----------+----------+------------+--------------------+--------------------+----------+----------+--------------------+
| 3825848|2015-09-23|         La Liga|Regular Season|  2015/2016|         5|20:00:00.000|          Levante UD|               Eibar|         2|         2|2023-02-21T15:19:...|
| 3825895|2015-09-23|         La Liga|Regular Season|  2015/2016|         5|22:00:00.000|          Las Palmas|             Sevilla|         2|         0|2023-02-21T15:19:...|
| 3825894|2016-05-01|         La Liga|Regular Season|  2015/2016|        36|18:15:00.000|RC Deportivo La C...|              G

In [ ]:
matches_df_flat.count()

3433